In [ ]:
import sys
import os

# Добавьте путь к папке, где находятся ваши модули
sys.path.append(os.path.abspath("C:/Users/klimm/Desktop/lab2/layers"))


# Импортируем нужные модули
import numpy as np
import matplotlib.pyplot as plt

from Conv2D import Conv2D
from Im2Col import Im2Col
# from layers.batchnorm import BatchNorm
# from layers.dense import Dense
# from layers.maxpool import MaxPool
# from layers.dropout import Dropout

from utils.load_mnist import load_mnist_data
from utils.split_train_validation import split_train_validation
from utils.one_hot_encode import one_hot_encode

from optimizers import adjust_learning_rate, adam_update
from losses import l2_regularization, softmax_loss
from activations import leaky_relu, softmax
from metrics import compute_accuracy



Загружаем MNIST и разделяем выборку

In [14]:

# Загружаем MNIST данные
X_train, Y_train, X_test, Y_test = load_mnist_data()

# Проверяем размерности исходных данных
print(f"Исходные данные:\nX_train: {X_train.shape}, Y_train: {Y_train.shape}")
print(f"X_test: {X_test.shape}, Y_test: {Y_test.shape}")

# Разделяем обучающую выборку на 70% для обучения и 30% для валидации
X_train, X_test, Y_train, Y_test = split_train_validation(X_train, Y_train)

# Проверяем размеры новых выборок
print(f"После разделения:\nX_train: {X_train.shape}, Y_train: {Y_train.shape}")
print(f"X_test: {X_test.shape}, Y_test: {Y_test.shape}")


Read magic number: 2051
Images: num_images=60000, rows=28, cols=28
Loaded 47040000 image pixels.
Read magic number: 2049
Read magic number: 2051
Images: num_images=10000, rows=28, cols=28
Loaded 7840000 image pixels.
Read magic number: 2049
Исходные данные:
X_train: (60000, 28, 28), Y_train: (60000,)
X_test: (10000, 28, 28), Y_test: (10000,)
После разделения:
X_train: (42000, 28, 28), Y_train: (42000,)
X_test: (18000, 28, 28), Y_test: (18000,)


In [ ]:
print(X_train[0])

Нормализуем и добавляем метки

In [15]:
# Добавляем ось каналов
X_train = X_train[..., np.newaxis]
X_test = X_test[..., np.newaxis]

# Нормализуем данные об изображениях в диапазон [0, 1]
X_train = X_train / 255.0
X_test = X_test / 255.0

# Кодируем метки
Y_train = one_hot_encode(Y_train)
Y_test = one_hot_encode(Y_test)

In [11]:
# Проверяем размерности и значения после нормализации
print(X_train.shape, Y_train.shape, X_test.shape, Y_test.shape)
print(np.min(X_train), np.max(X_train))  # Ожидается: 0.0 1.0
# Проверяем one-hot encoding
print(Y_train[0], Y_test[0])

(42000, 28, 28, 1) (42000, 10) (18000, 28, 28, 1) (18000, 10)
0.0 1.0
[0. 0. 0. 0. 0. 0. 0. 1. 0. 0.] [0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]


In [ ]:
# def im2col(input_images, kernel_size, stride=1, padding=0):
#     """
#     Преобразует батч изображений в формат столбцов для ускоренной свертки.
#     :param input_images: Входные изображения (batch_size, height, width, channels).
#     :param kernel_size: Размер фильтра (F_h, F_w).
#     :param stride: Шаг свёртки.
#     :param padding: Паддинг.
#     :return: Развёрнутая матрица.
#     """
#     batch_size, height, width, channels = input_images.shape

#     # Размеры фильтра
#     if isinstance(kernel_size, int):
#         kernel_height = kernel_width = kernel_size
#     else:
#         kernel_height, kernel_width = kernel_size

#     # Применяем паддинг, если он есть
#     if padding > 0:
#         input_images = np.pad(input_images, [(0, 0), (padding, padding), (padding, padding), (0, 0)], mode='constant', constant_values=0)

#     # Размеры выходного изображения для каждого изображения
#     out_h = (height + 2 * padding - kernel_height) // stride + 1
#     out_w = (width + 2 * padding - kernel_width) // stride + 1

#     # Используем as_strided для получения всех "окон" (patches)
#     shape = (batch_size, out_h, out_w, kernel_height, kernel_width, channels)
#     strides = (input_images.strides[0], stride * input_images.strides[1], stride * input_images.strides[2],
#                input_images.strides[1], input_images.strides[2], input_images.strides[3])

#     # Используем as_strided для создания 6D массива: (batch_size, out_h, out_w, kernel_height, kernel_width, channels)
#     patches = np.lib.stride_tricks.as_strided(input_images, shape=shape, strides=strides)

#     # Преобразуем 6D массив в 2D, где каждая колонка это развернутый патч (kernel_height * kernel_width * channels)
#     im2col_matrix = patches.reshape(batch_size, -1, out_h * out_w)

#     # Транспонируем результат, чтобы получить нужный формат
#     im2col_matrix = im2col_matrix.transpose(0, 2, 1).reshape(batch_size * out_h * out_w, -1)

#     return im2col_matrix


# def conv2d_im2col(input_images, filters, stride=1, padding=0, bias=None):
#     """
#     Реализация свёртки для батча изображений с использованием im2col и матричного умножения.
#     :param input_images: Входные изображения (N, H, W, C).
#     :param filters: Фильтры (K, F_h, F_w, C), где K - количество фильтров.
#     :param stride: Шаг свёртки.
#     :param padding: Паддинг.
#     :param bias: Смещение для каждого фильтра (K).
#     :return: Результат свертки (N, out_h, out_w, K).
#     """
#     N, H, W, C = input_images.shape  # N - количество изображений, H - высота, W - ширина, C - количество каналов
#     K, F_h, F_w, C_f = filters.shape  # K - количество фильтров, F_h - высота фильтра, F_w - ширина фильтра, C_f - количество каналов в фильтре (должно быть равно C)

#     # Проверка: количество каналов фильтра должно совпадать с количеством каналов изображения
#     if C != C_f:
#         raise ValueError("Количество каналов в изображении и фильтре должно совпадать")

#     # Размеры выходного изображения для каждого изображения
#     out_h = (H + 2 * padding - F_h) // stride + 1
#     out_w = (W + 2 * padding - F_w) // stride + 1

#     # Разворачиваем входные данные с помощью im2col
#     im2col_matrix = im2col(input_images, (F_h, F_w), stride, padding)

#     # Разворачиваем фильтры в одномерные векторы
#     filters_col = filters.reshape(K, -1)  # Размерность (K, F_h * F_w * C)

#     # Матричное умножение: фильтры * im2col_matrix
#     # im2col_matrix теперь имеет форму (N * out_h * out_w, F_h * F_w * C)
#     # filters_col имеет форму (K, F_h * F_w * C)
#     # Мы выполняем батч матричного умножения, и результат будет (N * out_h * out_w, K)
#     out_col = np.matmul(im2col_matrix, filters_col.T)  # Размерность (batch_size * out_h * out_w, K)

#     # Восстанавливаем размерность результата
#     out = out_col.reshape(N, out_h, out_w, K)

#     if bias is not None:
#         out += bias.reshape(1, 1, 1, K)

#     return out


In [ ]:
class CNN:
    def __init__(self, kernel_size, num_filters, num_classes, input_channels=1, learning_rate=0.001, beta1=0.9, beta2=0.999, epsilon=1e-6, dropout_rate = 0.4):
        self.kernel_size = kernel_size
        self.num_filters = num_filters
        self.num_classes = num_classes
        self.input_channels = input_channels
        self.learning_rate = learning_rate
        self.beta1 = beta1
        self.beta2 = beta2
        self.epsilon = epsilon
        self.dropout_rate = dropout_rate # Коэффициент dropout
        self.gamma = np.ones(self.num_filters)
        self.beta = np.zeros(self.num_filters)
        self.m_gamma = np.zeros_like(self.gamma)
        self.v_gamma = np.zeros_like(self.gamma)
        self.m_beta = np.zeros_like(self.beta)
        self.v_beta = np.zeros_like(self.beta)

        F_h, F_w = kernel_size

        # Инициализация для сверточных фильтров с использованием He-инициализации
        self.filters = np.random.randn(num_filters, F_h, F_w, input_channels) * np.sqrt(2. / (F_h * F_w * input_channels))

        self.bias = np.zeros((num_filters,))

        self.fc_input_size = (28 - F_h + 1) * (28 - F_w + 1) * num_filters
        # Инициализация для fully connected слоев
        self.W_fc = np.random.randn(self.fc_input_size, num_classes) * np.sqrt(2. / self.fc_input_size)

        self.b_fc = np.zeros((num_classes,))

        self.m_filters = np.zeros_like(self.filters)
        self.v_filters = np.zeros_like(self.filters)
        self.m_bias = np.zeros_like(self.bias)
        self.v_bias = np.zeros_like(self.bias)

        self.m_W_fc = np.zeros_like(self.W_fc)
        self.v_W_fc = np.zeros_like(self.W_fc)
        self.m_b_fc = np.zeros_like(self.b_fc)
        self.v_b_fc = np.zeros_like(self.b_fc)

        self.t = 0

    def forward(self, X, training=True):
        conv_out = Conv2D(X, filters=self.filters, stride=1, padding=0, bias=self.bias)
        conv_out_norm, cache = self.batch_norm_forward(conv_out)

        pooled_out, max_pool_cache = max_pool(conv_out_norm)
        cache['max_pool'] = max_pool_cache

        N, H, W, C = pooled_out.shape
        self.fc_input = conv_out_norm.reshape(N, H * W * C)

        fc_out = self.fully_connected(self.fc_input, self.W_fc, self.b_fc)
        fc_out_leaky_relu = self.leaky_relu(fc_out)

        if training:
            fc_out_leaky_relu = self.dropout_forward(fc_out_leaky_relu)

        y_pred = self.softmax(fc_out_leaky_relu)

        return y_pred, cache

    
    def backpropagate(self, X, y_true, dout, cache):
        grad_filters = np.zeros_like(self.filters)  # Градиент для фильтров
        grad_bias = np.zeros_like(self.bias)  # Градиент для биасов

        N, H, W, C = X.shape

        # Обратное распространение для max-pooling
        if 'max_pool' in cache:
            grad_pool = max_pooling_backward(dout, cache['max_pool'], pool_size=2, stride=2, padding=0)
            dout = grad_pool  # Обновляем dout для следующего слоя

        # Для полносвязанного слоя
        if dout.ndim == 2:
            grad_W_fc = np.dot(self.fc_input.T, dout)  # Градиенты для весов полносвязанного слоя
            grad_b_fc = np.sum(dout, axis=0)  # Градиенты к биасам полносвязанного слоя

            return grad_filters, grad_W_fc, grad_b_fc, None, None

        # Для свёрточного слоя
        elif dout.ndim == 4:
            dout_reshaped = dout.reshape(N, H, W, C)

            # Обратное распространение через BatchNorm
            dx, grad_gamma, grad_beta = self.batch_norm_backward(dout_reshaped, cache['batch_norm'])

            # Обратное распространение через сверточный слой
            grad_filters = np.sum(X * dx, axis=(0, 1), keepdims=True)  # Градиенты по фильтрам свертки

            # Возвращаем градиенты для свертки и BatchNorm
            return grad_filters, None, None, grad_gamma, grad_beta

        else:
            # Если количество измерений в dout не соответствует ожиданиям
            raise ValueError(f"Unexpected number of dimensions for dout: {dout.ndim}")



    def train(self, X_train, y_train, epochs=10, batch_size=64):
        """Обучение сети с использованием оптимизатора Adam."""
        num_samples = X_train.shape[0]
        for epoch in range(epochs):
            epoch_loss = 0
            epoch_acc = 0
            for start in range(0, num_samples, batch_size):
                # Получаем текущий батч
                end = min(start + batch_size, num_samples)
                X_batch = X_train[start:end]
                y_batch = y_train[start:end]

                # Прямой проход
                y_pred, cache = self.forward(X_batch)

                # Функция потерь и точность
                loss = self.softmax_loss(y_batch, y_pred)
                epoch_loss += loss
                acc = self.compute_accuracy(y_batch, y_pred)
                epoch_acc += acc

                # Обратное распространение
                dout = y_pred - y_batch
                grad_filters, grad_W_fc, grad_b_fc, grad_gamma, grad_beta = self.backpropagate(X_batch, y_batch, dout, cache)

                # Обновление параметров с использованием Adam
                self.adam_update(grad_W_fc, grad_b_fc, grad_filters, grad_gamma, grad_beta)

            # Adjust learning rate every 10 epochs
            self.adjust_learning_rate(epoch)

            # Вывод потерь и точности каждую эпоху
            print(f'Epoch {epoch+1}/{epochs}, Loss: {epoch_loss / (num_samples // batch_size):.4f}, Accuracy: {epoch_acc / (num_samples // batch_size):.4f}')


In [ ]:
# Пример инициализации и обучения
cnn = CNN(kernel_size=(5, 5), num_filters=32, num_classes=10, learning_rate=0.0001)
# Обучение модели
cnn.train(X_train, Y_train, epochs=1000, batch_size=256)

ValueError: too many values to unpack (expected 2)

In [ ]:
# Пример инициализации и обучения
cnn = CNN(kernel_size=(3, 3), num_filters=32, num_classes=10, learning_rate=0.0001)
# Обучение модели
cnn.train(X_train, Y_train, epochs=10, batch_size=256)

Epoch 1/10, Loss: 8.0281, Accuracy: 0.4645
Epoch 2/10, Loss: 8.7033, Accuracy: 0.4658
Epoch 3/10, Loss: 8.7301, Accuracy: 0.4711
Epoch 4/10, Loss: 8.7747, Accuracy: 0.4720
Epoch 5/10, Loss: 8.7773, Accuracy: 0.4761
Epoch 6/10, Loss: 8.9009, Accuracy: 0.4726
Epoch 7/10, Loss: 9.1026, Accuracy: 0.4718
Epoch 8/10, Loss: 9.1885, Accuracy: 0.4722
Epoch 9/10, Loss: 9.3865, Accuracy: 0.4711
Epoch 10/10, Loss: 9.4948, Accuracy: 0.4762


In [ ]:
#    def dropout_forward(self, x, training=True):
#         if not training:
#             return x  # Возвращаем без изменений на этапе тестирования
#         # Случайным образом выключаем нейроны
#         mask = (np.random.rand(*x.shape) < self.dropout_rate) / self.dropout_rate # Масштабируем веса на тестировании
#         return x* mask

import numpy as np

# def fully_connected(self, input_data, weights, bias):
#     return np.dot(input_data, weights) + bias

In [ ]:
# Пример инициализации и обучения
cnn = CNN(kernel_size=(3, 3), num_filters=32, num_classes=10, learning_rate=0.001)
# Обучение модели
cnn.train(X_train, Y_train, epochs=10, batch_size=64)


Epoch 1/10, Loss: 11.3275, Accuracy: 0.1022
Epoch 2/10, Loss: 9.1683, Accuracy: 0.1202
Epoch 3/10, Loss: 8.8478, Accuracy: 0.1246
Epoch 4/10, Loss: 9.0712, Accuracy: 0.1258
Epoch 5/10, Loss: 9.6075, Accuracy: 0.1270
Epoch 6/10, Loss: 10.4819, Accuracy: 0.1215
Epoch 7/10, Loss: 10.5304, Accuracy: 0.1231
Epoch 8/10, Loss: 10.5588, Accuracy: 0.1276


KeyboardInterrupt: 

In [ ]:

# @jit(nopython=True, parallel=True)
# def im2col_optimized_numba(input_images, kernel_size, stride=1, padding=0):
#     batch_size, height, width, channels = input_images.shape
#     if isinstance(kernel_size, int):
#         kernel_height = kernel_width = kernel_size
#     else:
#         kernel_height, kernel_width = kernel_size

#     if padding > 0:
#         input_images = np.pad(input_images, [(0, 0), (padding, padding), (padding, padding), (0, 0)], mode='constant', constant_values=0)

#     out_h = (height + 2 * padding - kernel_height) // stride + 1
#     out_w = (width + 2 * padding - kernel_width) // stride + 1

#     unfold = np.zeros((batch_size, kernel_height * kernel_width * channels, out_h * out_w))

#     for n in range(batch_size):
#         for i in range(0, out_h * stride, stride):
#             for j in range(0, out_w * stride, stride):
#                 patch = input_images[n, i:i + kernel_height, j:j + kernel_width, :]
#                 unfold[n, :, (i // stride) * out_w + (j // stride)] = patch.flatten()

#     return unfold


In [ ]:

# @jit(nopython=True, parallel=True)
# def im2col_optimized_numba(input_images, kernel_size, stride=1, padding=0):
#     batch_size, height, width, channels = input_images.shape
#     if isinstance(kernel_size, int):
#         kernel_height = kernel_width = kernel_size
#     else:
#         kernel_height, kernel_width = kernel_size

#     if padding > 0:
#         input_images = np.pad(input_images, [(0, 0), (padding, padding), (padding, padding), (0, 0)], mode='constant', constant_values=0)

#     out_h = (height + 2 * padding - kernel_height) // stride + 1
#     out_w = (width + 2 * padding - kernel_width) // stride + 1

#     unfold = np.zeros((batch_size, kernel_height * kernel_width * channels, out_h * out_w))

#     for n in range(batch_size):
#         for i in range(0, out_h * stride, stride):
#             for j in range(0, out_w * stride, stride):
#                 patch = input_images[n, i:i + kernel_height, j:j + kernel_width, :]
#                 unfold[n, :, (i // stride) * out_w + (j // stride)] = patch.flatten()

#     return unfold


In [ ]:
# class CNN:
#     def __init__(self, kernel_size, num_filters, num_classes, input_channels=1, learning_rate=0.001, beta1=0.9, beta2=0.999, epsilon=1e-6, dropout_rate = 0.4):
#         self.kernel_size = kernel_size
#         self.num_filters = num_filters
#         self.num_classes = num_classes
#         self.input_channels = input_channels
#         self.learning_rate = learning_rate
#         self.beta1 = beta1
#         self.beta2 = beta2
#         self.epsilon = epsilon
#         self.dropout_rate = dropout_rate # Коэффициент dropout
#         self.gamma = np.ones(self.num_filters)
#         self.beta = np.zeros(self.num_filters)
#         self.m_gamma = np.zeros_like(self.gamma)
#         self.v_gamma = np.zeros_like(self.gamma)
#         self.m_beta = np.zeros_like(self.beta)
#         self.v_beta = np.zeros_like(self.beta)

#         F_h, F_w = kernel_size

#         # Инициализация для сверточных фильтров с использованием He-инициализации
#         self.filters = np.random.randn(num_filters, F_h, F_w, input_channels) * np.sqrt(2. / (F_h * F_w * input_channels))

#         self.bias = np.zeros((num_filters,))

#         self.fc_input_size = (28 - F_h + 1) * (28 - F_w + 1) * num_filters
#         # Инициализация для fully connected слоев
#         self.W_fc = np.random.randn(self.fc_input_size, num_classes) * np.sqrt(2. / self.fc_input_size)

#         self.b_fc = np.zeros((num_classes,))

#         self.m_filters = np.zeros_like(self.filters)
#         self.v_filters = np.zeros_like(self.filters)
#         self.m_bias = np.zeros_like(self.bias)
#         self.v_bias = np.zeros_like(self.bias)

#         self.m_W_fc = np.zeros_like(self.W_fc)
#         self.v_W_fc = np.zeros_like(self.W_fc)
#         self.m_b_fc = np.zeros_like(self.b_fc)
#         self.v_b_fc = np.zeros_like(self.b_fc)

#         self.t = 0

#     def forward(self, X, training=True):
#         conv_out = Conv2D(X, filters=self.filters, stride=1, padding=0, bias=self.bias)
#         conv_out_norm, cache = self.batch_norm_forward(conv_out)

#         pooled_out, max_pool_cache = max_pool(conv_out_norm)
#         cache['max_pool'] = max_pool_cache

#         N, H, W, C = pooled_out.shape
#         self.fc_input = conv_out_norm.reshape(N, H * W * C)

#         fc_out = self.fully_connected(self.fc_input, self.W_fc, self.b_fc)
#         fc_out_leaky_relu = self.leaky_relu(fc_out)

#         if training:
#             fc_out_leaky_relu = self.dropout_forward(fc_out_leaky_relu)

#         y_pred = self.softmax(fc_out_leaky_relu)

#         return y_pred, cache

    
#     def backpropagate(self, X, y_true, dout, cache):
#         grad_filters = np.zeros_like(self.filters)  # Градиент для фильтров
#         grad_bias = np.zeros_like(self.bias)  # Градиент для биасов

#         N, H, W, C = X.shape

#         # Обратное распространение для max-pooling
#         if 'max_pool' in cache:
#             grad_pool = max_pooling_backward(dout, cache['max_pool'], pool_size=2, stride=2, padding=0)
#             dout = grad_pool  # Обновляем dout для следующего слоя

#         # Для полносвязанного слоя
#         if dout.ndim == 2:
#             grad_W_fc = np.dot(self.fc_input.T, dout)  # Градиенты для весов полносвязанного слоя
#             grad_b_fc = np.sum(dout, axis=0)  # Градиенты к биасам полносвязанного слоя

#             return grad_filters, grad_W_fc, grad_b_fc, None, None

#         # Для свёрточного слоя
#         elif dout.ndim == 4:
#             dout_reshaped = dout.reshape(N, H, W, C)

#             # Обратное распространение через BatchNorm
#             dx, grad_gamma, grad_beta = self.batch_norm_backward(dout_reshaped, cache['batch_norm'])

#             # Обратное распространение через сверточный слой
#             grad_filters = np.sum(X * dx, axis=(0, 1), keepdims=True)  # Градиенты по фильтрам свертки

#             # Возвращаем градиенты для свертки и BatchNorm
#             return grad_filters, None, None, grad_gamma, grad_beta

#         else:
#             # Если количество измерений в dout не соответствует ожиданиям
#             raise ValueError(f"Unexpected number of dimensions for dout: {dout.ndim}")



#     def train(self, X_train, y_train, epochs=10, batch_size=64):
#         """Обучение сети с использованием оптимизатора Adam."""
#         num_samples = X_train.shape[0]
#         for epoch in range(epochs):
#             epoch_loss = 0
#             epoch_acc = 0
#             for start in range(0, num_samples, batch_size):
#                 # Получаем текущий батч
#                 end = min(start + batch_size, num_samples)
#                 X_batch = X_train[start:end]
#                 y_batch = y_train[start:end]

#                 # Прямой проход
#                 y_pred, cache = self.forward(X_batch)

#                 # Функция потерь и точность
#                 loss = self.softmax_loss(y_batch, y_pred)
#                 epoch_loss += loss
#                 acc = self.compute_accuracy(y_batch, y_pred)
#                 epoch_acc += acc

#                 # Обратное распространение
#                 dout = y_pred - y_batch
#                 grad_filters, grad_W_fc, grad_b_fc, grad_gamma, grad_beta = self.backpropagate(X_batch, y_batch, dout, cache)

#                 # Обновление параметров с использованием Adam
#                 self.adam_update(grad_W_fc, grad_b_fc, grad_filters, grad_gamma, grad_beta)

#             # Adjust learning rate every 10 epochs
#             self.adjust_learning_rate(epoch)

#             # Вывод потерь и точности каждую эпоху
#             print(f'Epoch {epoch+1}/{epochs}, Loss: {epoch_loss / (num_samples // batch_size):.4f}, Accuracy: {epoch_acc / (num_samples // batch_size):.4f}')
